In [7]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import random

In [2]:
class CausalSelfAttention(nn.Module):

    def ___init__(self,config):
        super().__init__()
        assert config.n_embd %config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y

In [11]:

import pandas as pd
df = pd.read_csv("/Users/zhouchaoran/downloads/philosophy_data.csv")

target_column = "sentence_spacy"
sampled_df = df.sample(n=30000, random_state=42)[[target_column]]

In [9]:
with open('sampled_philosophy.txt', 'w', encoding='utf-8') as f:
    for sentence in sampled_df[target_column]:
        if random.random() < 0.1:
            f.write(sentence + ' \n<DEAD, REBORNING...>\n')
        else:
            f.write(sentence + '\n')



In [10]:
with open('sampled_philosophy.txt', 'r', encoding='utf-8') as d:
    sampled_content = d.read()

with open('one.txt', 'a', encoding='utf-8') as x:
    x.write('\n' + sampled_content)
